In [14]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset

isot_train = pd.read_csv('data/isot/preprocessed/train.csv')
isot_test = pd.read_csv('data/isot/preprocessed/test.csv')


kaggle_train = pd.read_csv('data/kaggle/preprocessed/train.csv')
kaggle_test = pd.read_csv('data/kaggle/preprocessed/test.csv')

isot_train

,text,label
0,saudi graft inquiry spreads beyond borders as ...,1
1,bernie sanders releases minidocumentary on fl...,0
2,un aims to open libyan transit center early ne...,1
3,japan says north korea not interested in meani...,1
4,afghan suspect arrested in germany over migran...,1
...,...,...
35913,clinton global initiative member to moderate f...,0
35914,sweden loses 14000 refugees slated for deporta...,0
35915,cuban invasion 7000 crossing border into us wi...,0
35916,trump threatens rude media whines that report...,0


In [15]:
def create_fake_news_model(data: pd.DataFrame):
    # Vérifier que le dataframe contient les bonnes colonnes
    if "text" not in data.columns or "label" not in data.columns:
        raise ValueError("Les colonnes 'text' et 'label' sont nécessaires dans le DataFrame")

    # Séparer les données en entraînement et validation
    train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)

    # Utilisation du tokenizer BERT pré-entrainé pour tokeniser le texte
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True)

    # Convertir les DataFrames en objets Dataset pour Hugging Face
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)

    # Appliquer la tokenisation
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    val_dataset = val_dataset.map(tokenize_function, batched=True)

    # Charger le modèle BERT pré-entrainé pour la classification binaire
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    # Définir les arguments d'entraînement
    training_args = TrainingArguments(
        output_dir='./results',          # Dossier où stocker les résultats
        num_train_epochs=3,              # Nombre d'époques d'entraînement
        per_device_train_batch_size=16,  # Taille des lots pour l'entraînement
        per_device_eval_batch_size=64,   # Taille des lots pour la validation
        warmup_steps=500,                # Nombre de pas pour le warmup
        weight_decay=0.01,               # Pénalité de poids pour régularisation
        logging_dir='./logs',            # Dossier pour stocker les logs
        logging_steps=10,
    )

    # Utilisation du Trainer pour l'entraînement et l'évaluation
    trainer = Trainer(
        model=model,                         # Le modèle BERT
        args=training_args,                  # Arguments d'entraînement
        train_dataset=train_dataset,         # Jeu d'entraînement
        eval_dataset=val_dataset,            # Jeu de validation
    )

    # Entraîner le modèle
    trainer.train()

    # Sauvegarder le modèle et le tokenizer
    model.save_pretrained('./fake_news_model')
    tokenizer.save_pretrained('./fake_news_model')

    # Retourner le modèle entraîné
    return model, tokenizer

In [16]:
# Créer et entraîner le modèle
model, tokenizer = create_fake_news_model(isot_train)

# Sauvegarder ou utiliser le modèle
print("Modèle créé et entraîné avec succès.")

Map: 100%|██████████| 7184/7184 [00:21<00:00, 337.59 examples/s]
model.safetensors:  10%|▉         | 41.9M/440M [00:56<08:55, 744kB/s]


KeyboardInterrupt: 